<a href="https://colab.research.google.com/github/b3GOR/Flang/blob/main/FLang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Backend


In [ ]:
%%capture
!pip install transformers accelerate

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch
from google.colab import files

In [ ]:
model = "b3G0R/FLang"
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)


Загрузка словаря

In [ ]:
uploaded = files.upload()
file_names = list(uploaded.keys())
first_file_name = file_names[0]
with open(first_file_name, 'r') as file:
  w = file.read().split()
  dict = ', '.join(w)



In [ ]:
# @title Генератор текста к словам
topic = "life" # @param {type:"string"}
sentence = "4" # @param {type:"string"}
level = "B1" # @param ["A1", "A2", "B1", "B2", "C1", "C2"]
words = "man, woman, work, raise" # @param {type:"string"}


prompt = f"write a text on English of {sentence} sentence about {topic} topic, but use these words:{words}"
#The text must be written at an English level {level}.'''
SYS = f'''You are an English teacher, and your task is to compose a well-written English text that demonstrates strong grammar, style, and semantics.
The text should be tailored to an English proficiency level of {level}.
The text must consist of only these words ({dict}) and these words {words}'''


sequences = pipeline(
    f'[INST] <<SYS>> {SYS}  <</SYS>> {prompt} [/INST]',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=800,
)

generated_text = sequences[0]['generated_text']

#while "[INST]" in generated_text:
#    start_index = generated_text.find("[INST]")
#    end_index = generated_text.find("[/INST]", start_index)

#    if start_index != -1 and end_index != -1:
 #       generated_text = generated_text[:start_index] + generated_text[end_index + 7:]
  #  else:
   #     break

# Разделяем предложения символом новой строки
generated_text = generated_text.replace(".", ".\n")

print(generated_text)